# Clase 1-2 Introducción a Regresión Logística y Cross Validation

## Regresión logística.

La regresión logística es un algoritmo de clasificación utilizado para predecir la **probabilidad de que un evento ocurra**. A diferencia de la regresión lineal, que produce valores continuos, la regresión logística produce **valores entre 0 y 1**, que se interpretan como probabilidades. Se utiliza comúnmente en problemas de **clasificación binaria**, donde el resultado puede ser **verdadero/falso**, **sí/no**, etc. El algoritmo utiliza la función logística para modelar la relación entre las variables de entrada y la probabilidad del evento.

In [ ]:
Data.annual_inc.hist(bins=100, density=True)
plt.title("Annual Income Distribution")
plt.xlabel("Annual Income")

<img src='anual_income_dist_1.jpeg'>

In [ ]:
np.log(Data.annual_inc).hist(bins=100, density=True)
plt.title("Annual Income Distribution")
plt.xlabel("Annual Income")

Aplicando logaritmo a los datos

<img src='anual_income_dist_2.jpeg'>

## Cross Validation

La validación cruzada (cross-validation) es una técnica utilizada para **evaluar el rendimiento de un modelo de aprendizaje automático**. Consiste en dividir el conjunto de datos en subconjuntos de entrenamiento y prueba de manera iterativa, de modo que cada subconjunto se utilice tanto para entrenar como para probar el modelo. Esto **permite obtener una evaluación más robusta del rendimiento del modelo** al promediar los resultados de múltiples particiones. La validación cruzada es **especialmente útil para evitar el sobreajuste (overfitting)** y para seleccionar hiperparámetros de manera más precisa.

In [ ]:
skf = StratifiedKFold(n_splits=5)
for k, (train_index, test_index) in enumerate( skf.split(X, y) ):
    plt.plot(train_index, [k+1 for _ in train_index], ".")
plt.ylim(0,6)
plt.ylabel("FOLD")
plt.title("CROSS VALIDATION FOLDS")

<img src='cross_validation.jpeg'>

Este fragmento de código define una función llamada "compute_AUC" que calcula el área bajo la curva ROC (AUC) para un modelo de regresión logística. Aquí está el desglose de lo que hace:

Toma como entrada las características (X), las salidas (y) y los índices de división del conjunto de datos (train_index, test_index).
Divide el conjunto de datos en conjuntos de entrenamiento y prueba utilizando los índices proporcionados.
Ajusta un modelo de regresión logística a los datos de entrenamiento llamando a la función "fit_logistic_regression".
Calcula las probabilidades predichas para las muestras de prueba utilizando el modelo ajustado, y extrae las probabilidades de pertenecer a la clase positiva.
Calcula la tasa de falsos positivos (fpr), la tasa de verdaderos positivos (tpr) y el umbral (no utilizado) utilizando las etiquetas verdaderas y las probabilidades predichas.
Calcula el área bajo la curva (AUC) utilizando las tasas de fpr y tpr.
Devuelve el valor del AUC, así como las tasas de fpr y tpr.
En resumen, esta función se encarga de calcular el AUC y las curvas ROC para un modelo de regresión logística en un conjunto de datos dado

## Que es AUC y ROC

El AUC (Area Under the Curve) es una métrica que se utiliza para evaluar la calidad de un modelo de clasificación binaria. Especifica la capacidad del modelo para discriminar entre las clases positiva y negativa. El AUC mide el área bajo la curva ROC (Receiver Operating Characteristic), que es un gráfico de la tasa de verdaderos positivos (eje y) frente a la tasa de falsos positivos (eje x) a varios umbrales de clasificación.

En resumen, el AUC es una medida de la capacidad de discriminación de un modelo de clasificación binaria, y la curva ROC es una representación gráfica de esta capacidad. Un AUC más alto (más cercano a 1) indica un mejor rendimiento del modelo en la clasificación binaria.


In [ ]:
def compute_AUC(X, y, train_index, test_index):
    """
    feature/output: X, y
    dataset split: train_index, test_index
    """
    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_test, y_test = X.iloc[test_index], y.iloc[test_index]

    clf = fit_logistic_regression(X_train, y_train)
    default_proba_test = clf.predict_proba(X_test)[:,1]  
    fpr, tpr, _ = roc_curve(y_test, default_proba_test)
    auc_score = auc(fpr, tpr)
    return auc_score, fpr, tpr

In [ ]:
def cross_validation_AUC(X,y, nfold=10):
    """
    use a n-fold cross-validation for computing AUC estimates
    """
    skf = StratifiedKFold(n_splits=nfold)  #create a cross-validation splitting
    auc_list = [] #this list will contain the AUC estimates associated with each fold
    for k, (train_index, test_index) in enumerate( skf.split(X, y) ):
        auc_score, _, _ = compute_AUC(X, y, train_index, test_index)
        auc_list.append(auc_score)
    return auc_list

# Clase 5-6 Naive Bayes - Decision Tree - k-means

> Nota: Openpyxl es una libreria que permite manipular archivos de Microsoft Excel: `from openpyxl import load_workbook # Needed to open XLSX files`
> Nota2: La librería "mglearn" es una herramienta de visualización y utilidades para trabajar con algoritmos de aprendizaje automático en Python. Proporciona funciones y conjuntos de datos que son útiles para visualizar y comprender conceptos clave en aprendizaje automático, como regresión, clasificación, agrupación, selección de modelos y más. La librería "mglearn" es útil para educación, experimentación y visualización de resultados en aprendizaje automático. `! pip install mglearn;`


El método consta de 3 pasos amplios, que se pueden resumir de la siguiente manera:

1. Inicialización. Para comenzar, se deben seleccionar $k$ puntos (no necesariamente cualquiera de los puntos de datos, solo puntos en el mismo espacio dimensional) como los centroides "iniciales".
2. Asignación. Cada punto de datos se asigna al grupo correspondiente al centroide más cercano a él (generalmente según la distancia euclidiana estándar).
3. Actualización. Una vez que todos los puntos de datos se han asignado a sus respectivos grupos, se calcula un nuevo centroide para cada grupo tomando la media de todos los puntos de ese grupo.

Luego se repiten los pasos 2 y 3 hasta que los grupos ya no cambien.



Este código implementa el algoritmo de k-means para la agrupación de datos. Aquí está el desglose de lo que hace:

Se establece el número de clusters (k) en 4.
Se inicializan las posiciones iniciales de los centroides de los clusters de forma aleatoria dentro de un rango específico.
Se imprime la posición inicial de los centroides.
Se define un mapeo de colores para visualizar los clusters.
Se define una función llamada "assignment" que asigna cada punto de datos al centroide más cercano y asigna un color a cada punto basado en el centroide al que pertenece.
Se aplica la función "assignment" a los datos normalizados y se muestra una vista previa de los datos asignados a los centroides.
Se realiza una visualización de dispersión de los datos, con los puntos coloreados según su asignación a los centroides, y se muestran los centroides como puntos adicionales en el gráfico.
En resumen, el código implementa el algoritmo de k-means para agrupar datos y visualiza los resultados de la agrupación.

In [ ]:
# Now we will begin our k-means method:

# assign how many clusters (k-number) you would like to have
k = 4

# Initialization of cluster means 
centroids = {i+1: [np.random.choice([-1,-0.5,0.25,0.5]), np.random.choice([-1,-0.5,0.25,0.5])]
    for i in range(k)
}

print(f"Initial Centroids (age, income) are: {centroids}")

color_map = {1: 'r', 2: 'g', 3: 'b' , 4:'c', 5:'y'}

def assignment(df, centroids):
    tmp = df.copy()
    for i in centroids.keys():
        tmp[f"distance_from_{i}"] = (np.sqrt(
            (df['income'] - centroids[i][0])*2 + (tmp['claims'] - centroids[i][1]) * 2)
        ).round(2)
        
    centroid_distance_cols = [f"distance_from_{i}" for i in centroids.keys()]
    
    tmp['closest'] = tmp.loc[:, centroid_distance_cols].idxmin(axis=1)
    tmp['closest'] = tmp['closest'].map(lambda x: int(x.lstrip('distance_from_')))
    tmp['color'] = tmp['closest'].map(lambda x: color_map[x])
    
    return tmp

df_centroids = assignment(df_norm, centroids)
print(df_centroids.head())

plt.figure(figsize=(6, 6))
plt.scatter(df_centroids['income'], df_centroids['claims'], color=df_centroids['color'], alpha=0.5, edgecolor='k')

for i in centroids.keys():
    plt.scatter(*centroids[i], color=color_map[i])
    
plt.xlim(-1.5, 2.5)
plt.xlabel('Income')

plt.ylim(-1.5, 2.5)
plt.ylabel('Claims')

plt.show()

Este fragmento de código implementa la actualización de los centroides en el algoritmo de k-means. Aquí está el desglose de lo que hace:

Se realiza una copia profunda de los centroides actuales para conservar los valores anteriores.
Se define una función llamada "update" que calcula los nuevos centroides para cada cluster, tomando la media de las coordenadas de los puntos asignados a cada cluster.
Se actualizan los centroides llamando a la función "update" y se almacenan los nuevos valores.
Se realiza una visualización de dispersión de los datos, con los puntos coloreados según su asignación a los centroides, y se muestran los nuevos centroides como puntos adicionales en el gráfico. Además, se agregan flechas que muestran la dirección y magnitud del movimiento de los centroides desde su posición anterior hacia la nueva posición.
En resumen, este código implementa el paso de actualización de los centroides en el algoritmo de k-means y visualiza el movimiento de los centroides en el espacio de características.

In [ ]:
# Update Step: In this step, we are taking updating the mean, by now calculating it based on
# all the data points which were assigned to that k-th cluster.

# Copy previous values and make a new entry called old_centroids. 
import copy

old_centroids = copy.deepcopy(centroids)

# Function to update the k-th clusters. 
def update(k):
    for i in centroids.keys():
        centroids[i][0] = np.mean(df_centroids[df_centroids['closest'] == i]['income'])
        centroids[i][1] = np.mean(df_centroids[df_centroids['closest'] == i]['claims'])
    return k

centroids = update(centroids)
    
plt.figure(figsize=(5, 5))
ax = plt.axes()

plt.scatter(df_centroids['income'], df_centroids['claims'], color=df_centroids['color'], alpha=0.5, edgecolor='k')

for i in centroids.keys():
    plt.scatter(*centroids[i], color=color_map[i])
    
plt.xlim(-1.5, 2.5)
plt.ylim(-1.5, 2.5)

for i in old_centroids.keys():
    old_x = old_centroids[i][0]
    old_y = old_centroids[i][1]
    
    dx = (centroids[i][0] - old_centroids[i][0]) * 0.75
    dy = (centroids[i][1] - old_centroids[i][1]) * 0.75
    
    ax.arrow(old_x, old_y, dx, dy, head_width=0.05, head_length=0.03, fc=color_map[i], ec=color_map[i])
    
plt.show()

Este fragmento de código implementa la iteración del algoritmo de k-means hasta que no se realicen nuevas asignaciones de puntos a los centroides. Aquí está el desglose de lo que hace:

Se realiza la asignación de los puntos a los centroides más cercanos llamando a la función "assignment" con los centroides actuales.
Se inicia un bucle "while True" que continúa iterando hasta que no se realicen nuevas asignaciones de puntos a los centroides.
En cada iteración del bucle, se realizan los siguientes pasos: a. Se guarda una copia de las asignaciones más recientes de puntos a los centroides. b. Se actualizan los centroides llamando a la función "update" con los centroides actuales. c. Se vuelve a realizar la asignación de puntos a los centroides llamando a la función "assignment" con los centroides actualizados. d. Se verifica si las asignaciones más recientes son iguales a las asignaciones anteriores. Si son iguales, se rompe el bucle y se detiene la iteración.
Se realiza una visualización de dispersión de los datos, con los puntos coloreados según su asignación a los centroides, y se muestran los centroides actualizados como puntos adicionales en el gráfico.
En resumen, este código implementa el bucle de iteración del algoritmo de k-means, que actualiza repetidamente las asignaciones de puntos a centroides y los centroides mismos hasta que no se realicen cambios en las asignaciones.

In [ ]:
# Iterative procedure: Reassign new weights 
df_centroids = assignment(df_centroids, centroids)

# Continue doing this until no new assignments have been made 
while True:
    closest_centroids = df_centroids['closest'].copy(deep=True)
    centroids = update(centroids)
    df_centroids = assignment(df_centroids, centroids)
    
    if closest_centroids.equals(df_centroids['closest']):
        break

plt.figure(figsize=(6, 6))
plt.scatter(df_centroids['income'], df_centroids['claims'], color=df_centroids['color'], alpha=0.5, edgecolor='k')

for i in centroids.keys():
    plt.scatter(*centroids[i], color=color_map[i], edgecolor='k')
    
plt.xlim(-1.5, 2.5)
plt.xlabel('Income [normalized]')

plt.ylim(-1.5, 2.5)
plt.ylabel('Claims [normalized]')

plt.show()